In [34]:
from datetime import date, datetime, time,timedelta
import pandas as pd
import numpy as np

In [35]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:

from sklearn.ensemble import RandomForestClassifier

# from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

In [38]:
def dec_tree(X,Y,depth,crit):

    clf = DecisionTreeClassifier(max_depth=depth,
#                                  criterion=crit,
                                 max_features=crit,
                                    class_weight='balanced')
    
    #обучаемся на тест_трейне, для оценки точности методов
    a=cross_val_score(clf, X,Y, cv=7, scoring='recall')

    print([crit,depth,a.mean()])

In [39]:
def dec_rf(X,Y,depth,crit,estim):

    clf = RandomForestClassifier(max_depth=depth,
#                                  criterion=crit,
                                 max_features=crit,
                                 n_estimators=estim,
                                class_weight='balanced',
                                random_state=32,)
    
    #обучаемся на тест_трейне, для оценки точности методов
    a=cross_val_score(clf, X,Y, cv=7, scoring='recall')

    print([crit,depth,estim,a.mean()])

In [40]:
def dec_log(X,Y,depth,estim):

    clf = LogisticRegression(random_state=32,max_iter=1000,penalty='elasticnet',solver='saga',class_weight='balanced',l1_ratio=estim,C=depth)
    
    #обучаемся на тест_трейне, для оценки точности методов
    a=cross_val_score(clf, X,Y, cv=5, scoring='recall')

    print([depth,estim,a.mean()])

In [41]:
df=pd.read_excel(r'C:\Users\User\Documents\Обучающая выборка4.xlsx',sheet_name='train')

In [42]:
holyday=pd.read_excel(r'E:\Data\Новгород\Праздники.xlsx')

In [43]:
col_bin=['Погибло2','Погибло детей2','Ранено2','Ранено детей2']

place_col=['Погибло%место','Погибло детей%место','Ранено%место','Ранено детей%место']

type_col=['Погибло%вид','Погибло детей%вид','Ранено%вид','Ранено детей%вид']

In [44]:
df[col_bin]=df[['Погибло','Погибло детей','Ранено','Ранено детей']]>=1 # Преобразование в бинарную классификацию

stat_type=df[['Вид ДТП']+col_bin].groupby(by=['Вид ДТП'], as_index=False).sum()

stat_type[['Погибло%вид','Погибло детей%вид','Ранено%вид','Ранено детей%вид']]=stat_type[col_bin]/df[['Вид ДТП','Ранено']].groupby(['Вид ДТП']).count().values

stat_place=df.loc[~df['Вид ДТП'].isin(['Наезд на пешехода','Наезд на велосипедиста','Падение пассажира','Опрокидывание']),['Место2']+col_bin].groupby(by=['Место2'], as_index=False).sum()

stat_place[place_col]=stat_place[col_bin]/df[['Место2','Ранено']].groupby(['Место2']).count().values

stat_all=df[col_bin].sum()/df[col_bin].count()

In [45]:
stat_all

Погибло2          0.009645
Погибло детей2    0.000335
Ранено2           0.126974
Ранено детей2     0.021722
dtype: float64

In [46]:
def preproc(df,holyday,place_col,type_col,stat_type,stat_place,stat_all):
    
    df['Дата']=pd.to_datetime(df['Дата'])

    df['Month']=df['Дата'].dt.month

    df['Week']=df['Дата'].dt.isocalendar().week

    df['Weekday']=df['Дата'].dt.weekday

    df['Трасса']=df['Улица'].isna()
    
    df[['Rest','isMonday','isFriday']]=0
    

    df.loc[(df['Дата'].isin(holyday['date']))| (df['Weekday'].isin([5,6])),'Rest']=1
    
    
    df.loc[df['Weekday']==0,'isMonday']=1
    
    df.loc[df['Weekday']==4,'isFriday']=1
    

    #Time
    df['Час']=pd.to_datetime(df['Время'].astype(str)).dt.hour
    
    for it_hour in range(24):
        df['H{}'.format(it_hour)]=0
        df.loc[df['Час']==it_hour,'H{}'.format(it_hour)]=1
        
    df.loc[df['Час']<12,'Час']=df.loc[df['Час']<12,'Час']+24
        
    #Month
    for it_mon in range(1,13):
        df['M{}_1'.format(it_mon)]=0
        df['M{}_2'.format(it_mon)]=0
        
        df.loc[(df['Month']==it_mon)&(df['Дата'].dt.day<=15),'M{}_1'.format(it_mon)]=1
        df.loc[(df['Month']==it_mon)&(df['Дата'].dt.day>15),'M{}_2'.format(it_mon)]=1
        
    #Weekday
    
    for it_week in range(7):
        df['WD{}'.format(it_week)]=0
        df.loc[df['Weekday']==it_week,'WD{}'.format(it_week)]=1
    
    
    df=df.merge(stat_type[['Вид ДТП']+type_col],on='Вид ДТП',how='left').fillna(dict(zip(type_col,stat_all.values)))

    df=df.merge(stat_place[['Место2']+place_col],on='Место2',how='left').fillna(dict(zip(place_col,stat_all.values)))

    return(df)

In [47]:
train=preproc(df,holyday,place_col,type_col,stat_type,stat_place,stat_all)

test_data=preproc(pd.read_excel(r'E:\Data\test_place.xlsx'),
                      holyday,place_col,type_col,stat_type,stat_place,stat_all).set_index('id')

In [48]:
# Спрогнозируем Раненых

# Подготовим выборку для обучения
col_x=['Трасса', 'Rest', 'isFriday', 'isMonday','Ранено%вид', 'Ранено%место',
       'H0', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7',
       'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17',
       'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'M1_1', 'M1_2', 'M2_1',
       'M2_2', 'M3_1', 'M3_2', 'M4_1', 'M4_2', 'M5_1', 'M5_2', 'M6_1', 'M6_2',
       'M7_1', 'M7_2', 'M8_1', 'M8_2', 'M9_1', 'M9_2', 'M10_1', 'M10_2',
       'M11_1', 'M11_2', 'M12_1', 'M12_2','WD0','WD1', 'WD2', 'WD3', 'WD4', 'WD5', 'WD6']

col_y='Ранено2'

X,Y=train[col_x].fillna(0),train[col_y].fillna(0)

In [27]:
# Найдем оптимальные параметры для алгоритма
for it_depth in [3,4,5]:

    for it_crit in ['sqrt','log2']:
#         'log2',
        for it_estim in [5,15,50,150]:

            dec_rf(X,Y,it_depth,it_crit,it_estim)

['sqrt', 3, 5, 0.26728714369957307]


KeyboardInterrupt: 

In [49]:
          
# Ранен прогноз на тестовой выборке
clf=RandomForestClassifier(max_depth=4,n_estimators=5,max_features='log2',random_state=32,class_weight = 'balanced') #z4 0.5245

#обучаемся на всем множестве данных
clf.fit(X,Y)

test_data[col_y]=[x[1]>0.507 for x in clf.predict_proba(test_data[X.columns].fillna(0))]

# recall_score(Y,np.array([x[1]>=0.507 for x in clf.predict_proba(X.fillna(0))]),average=None).mean()

In [50]:
# Спрогнозируем Погибших

# Подготовим выборку для обучения
col_x=['Трасса', 'Rest', 'Погибло%вид', 'Погибло%место','Ранено%вид', 'Ранено%место',
       'H0', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7',
       'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17',
       'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'M1_1', 'M1_2', 'M2_1',
       'M2_2', 'M3_1', 'M3_2', 'M4_1', 'M4_2', 'M5_1', 'M5_2', 'M6_1', 'M6_2',
       'M7_1', 'M7_2', 'M8_1', 'M8_2', 'M9_1', 'M9_2', 'M10_1', 'M10_2',
       'M11_1', 'M11_2', 'M12_1', 'M12_2','WD0','WD1', 'WD2', 'WD3', 'WD4', 'WD5', 'WD6']


col_y='Погибло2'

X,Y=train[col_x].fillna(0),train[col_y].fillna(0)


In [ ]:
# Найдем оптимальные параметры для алгоритма
for it_depth in [3,4,5]:

    for it_crit in ['sqrt','log2',None]:
#         'log2',
        for it_estim in [5,15,50,150]:
            dec_rf(X,Y,it_depth,it_crit,it_estim)

In [51]:
clf=RandomForestClassifier(max_depth=3,n_estimators=150,max_features=None,random_state=32,class_weight = 'balanced') 

#обучаемся на всем множестве данных
clf.fit(X,Y)

# df_z01[col]=[x[1] for x in clf.predict_proba(X)]
    
test_data[col_y]=[x[1]>=0.5 for x in clf.predict_proba(test_data[X.columns].fillna(0))]*test_data['Ранено2']

# Если Нет Раненных значит нет погибших

In [68]:
# Спрогнозируем Ранено детей

# Подготовим выборку для обучения
col_x=['Трасса', 'Rest', 'Ранено детей%вид', 'Ранено детей%место',
       'H0', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7',
       'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17',
       'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'M1_1', 'M1_2', 'M2_1',
       'M2_2', 'M3_1', 'M3_2', 'M4_1', 'M4_2', 'M5_1', 'M5_2', 'M6_1', 'M6_2',
       'M7_1', 'M7_2', 'M8_1', 'M8_2', 'M9_1', 'M9_2', 'M10_1', 'M10_2',
       'M11_1', 'M11_2', 'M12_1', 'M12_2','WD0','WD1', 'WD2', 'WD3', 'WD4', 'WD5', 'WD6']

col_y='Ранено детей2'

X,Y=train.loc[train['Ранено2']>=1,col_x].fillna(0),train.loc[train['Ранено2']>=1,col_y].fillna(0)

In [69]:
# Найдем оптимальные параметры для алгоритма
for it_depth in [0.5,0.75,1]:
#         'log2',
    for it_estim in [0,0.25,0.5,0.75,1]:

        dec_log(X,Y,it_depth,it_estim)

[0.5, 0, 0.5147725392886684]
[0.5, 0.25, 0.512191894127378]
[0.5, 0.5, 0.5109346567411083]
[0.5, 0.75, 0.5083870967741935]
[0.5, 1, 0.5096856906534326]
[0.75, 0, 0.5186435070306038]
[0.75, 0.25, 0.5147808105872621]
[0.75, 0.5, 0.5135070306038048]
[0.75, 0.75, 0.5083705541770057]
[0.75, 1, 0.5019437551695616]
[1, 0, 0.5225144747725393]
[1, 0.25, 0.5160711331679073]
[1, 0.5, 0.5135153019023987]
[1, 0.75, 0.5109429280397022]
[1, 1, 0.5032423490488006]


In [63]:
# Ранено детей
clf=LogisticRegression(random_state=32,
                   max_iter=100,
                   penalty='elasticnet',
                   solver='saga',
                   class_weight='balanced',
                   l1_ratio=0,
                   C=1)
    

#обучаемся на всем множестве данных
clf.fit(X,Y)

# df_z01[col]=[x[1] for x in clf.predict_proba(X)]
    
test_data[col_y]=[x[1]>0.5 for x in clf.predict_proba(test_data[X.columns].fillna(0))]*test_data['Ранено2']

In [64]:
(test_data[['Ранено2', 'Ранено детей2','Погибло2']]*1).head(100)

,Ранено2,Ранено детей2,Погибло2
id,,,
490078911,0,0,0
490055448,0,0,0
490054440,0,0,0
490037847,0,0,0
490037049,0,0,0
...,...,...,...
490040461,0,0,0
490013825,0,0,0
490072230,0,0,0


In [67]:
(test_data[['Ранено2', 'Ранено детей2','Погибло2']]*1).to_excel(r'E:\Data\test_fil70.xlsx')

In [66]:
test_data['Ранено детей2'].sum()

621